In [1]:
import json
import pandas as pd
from os import listdir
from os.path import isfile, join

import vcd.core as core


In [2]:
!pip install pandas

In [3]:
#filename='gA_5_s1_2019-03-08T10;57;00+01;00_rgb_ann_distraction'
#filename='gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_distraction'
#filename='gA_5_s3_2019-03-13T09;10;35+01;00_rgb_ann_distraction'
#filename='gA_5_s4_2019-03-22T11;28;00+01;00_rgb_ann_distraction'
#data_dir = './data'
#datafiles = sorted([f for f in listdir(data_dir) if isfile(join(mypath, f))])
#datafiles


In [4]:
#datafile = 'gA_5_s1_2019-03-08T10;57;00+01;00_rgb_ann_distraction.json'
#fname = datafile[:datafile.find('.')]
#fname

In [5]:

#for datafile in datafiles:
    #Load a VCD filex
#    fname = datafile[:datafile.find('.')]
#    myVCD = core.VCD(data_dir + fname + ".json")

#    df=pd.DataFrame()
#    parse_frame_intervals()
#    parse_objects_data()

# Access data directly
#metadata = myVCD.data['openlabel']['metadata']

# Modify data directly
#myVCD['openlabel']['objects'][3]['type'] = "#Car"

# Serialize
#stringified_vcd = myVCD.stringify(pretty=False, validation=True)


In [6]:
#json_file_path = '/Users/suresh/dev/dmd/gA_5_s1_2019-03-08T10;57;00+01;00_rgb_ann_distraction.json'
#with open(json_file_path, 'r') as j:
#     contents = json.loads(j.read())
#contents
#df = pd.json_normalize(contents)
#df_frames = pd.json_normalize(contents['frames'])
#df_frames.head()

In [7]:
#df_objects = pd.json_normalize(contents['vcd']['objects'])
#df_objects.head()
#t2 = df_objects.transpose()
#t2

In [8]:
move_map = {
    "gaze_on_road/looking_road": False,
    "driver_actions/safe_drive": False,
    "gaze_on_road/not_looking_road": False,
    "hands_using_wheel/both": False,
    "hands_using_wheel/only_left": False,
    "driver_actions/change_gear": False,
    "hand_on_gear/hand_on_gear": False,
    "driver_actions/reach_side": True,
    "driver_actions/hair_and_makeup": True,
    "driver_actions/phonecall_right": True,
    "driver_actions/phonecall_left": True,
    "talking/talking": True,
    "driver_actions/texting_right": True,
    "driver_actions/phone": True,
    "driver_actions/drinking": True,
    "driver_actions/unclassified": False,
    "hands_using_wheel/none": False,
    "driver_actions/texting_left": True,
    "hands_using_wheel/only_right": False,
    "driver_actions/texting_left": True,
    "hand waving": True,
    "hair flowing": True,
    "scarf flying" : True
}

In [9]:
#df=pd.DataFrame()
#df

outdir = "./csv5/"

frame_end=0
frame_start=0
df=pd.DataFrame()
fname=''
movement=[]
fname_act = {}
fname_frames = {}

## Frame Intervals

In [10]:
def parse_frame_intervals():
    ## get_frame_intervals
    global frame_end
    global movement
    frame_intervals = myVCD.get_frame_intervals().get_dict()
    frame_start = frame_intervals[0]['frame_start']
    frame_end = frame_intervals[0]['frame_end']
    #print(frame_start, frame_end)

    df['frames'] = [i for i in range(frame_start,frame_end)]
    fname_frames[fname] = frame_end
    #df


In [11]:
def fill_frame_data(frame_data, obj):
    global movement
    name = obj['name']
    obj_type = obj['type']
    #print(name, obj_type)
    ##
    frame_intervals = obj['frame_intervals']
    for frame_interval in frame_intervals:
        obj_frame_start = frame_interval['frame_start']
        obj_frame_end = frame_interval['frame_end']
        # print(obj_frame_start, obj_frame_end)
        for i in range(obj_frame_start, obj_frame_end):
            frame_data[i] = True
            if obj_type in move_map and move_map[obj_type] == True:
                if fname not in fname_act:
                    fname_act[fname] = []
                fname_act[fname].append(obj_type)
                movement[i] = True
            else:
                movement[i] = False

    return frame_data
    

## Objects

In [12]:

def get_driver_data(object):
    name = object['name']
    obj_type = object['type']

    ##
    frame_data=[None] * frame_end
    frame_data = fill_frame_data(frame_data, object)
    ##
    object_data = object['object_data']
    #print(object_data)
    
    age = object_data['num'][0]['val']
    gender = object_data['text'][0]['val']
    experience = object_data['text'][1]['val']
    drive_freq = object_data['text'][2]['val']
    glass = object_data['boolean'][0]['val']
    
    #print(name, frame_start, frame_end, age, gender, experience, drive_freq,glass)
    
    df[obj_type] = frame_data
    df[obj_type+"_fname"] = [fname for i in range(frame_start,frame_end)]
    df[obj_type+"_name"] = [name for i in range(frame_start,frame_end)]
    df[obj_type+"_age"] = [age for i in range(frame_start,frame_end)]
    df[obj_type+"_experience"] = [experience for i in range(frame_start,frame_end)]
    df[obj_type+"_drive_freq"] = [drive_freq for i in range(frame_start,frame_end)]
    df[obj_type+"_glass"] = [glass for i in range(frame_start,frame_end)]
    #df.head()
    
    #print(name)
    
def get_bottle_data(object):
    name = object['name']
    obj_type = object['type']
    ##
    frame_data=[None] * frame_end
    frame_data = fill_frame_data(frame_data, object)
    df[obj_type] = frame_data

    
    

In [13]:
def parse_objects_data():
    ## Objects
    objects = myVCD.get_objects()
    for idx,object in objects.items():
        # print(object)
        if object['type'] == 'driver':
            driver_data = get_driver_data(object)
        #if object['type'] == 'bottle':
        get_bottle_data(object)

    df

In [14]:
#df.iloc[[1325,1326],:]

## Context

In [15]:
def get_contexts(context):
    name = context['name']
    ctype = context['type']
    context_data_s = context['context_data']['text']
    for context_data in context_data_s:
        if 'weather' in context_data['name']:
            weather = context_data['val']
            df["ctx_weather"] = [weather for i in range(frame_start,frame_end)]
        if 'setup' in context_data['name']:
            setup = context_data['val']
            df["ctx_setup"] = [setup for i in range(frame_start,frame_end)]

    print(name, ctype, weather, setup)

In [16]:
def parse_contexts():
    ## contexts
    contexts = myVCD.get_contexts()
    for idx,context in contexts.items():
        get_contexts(context)
    df

## actions

In [17]:
def parse_actions():
    actions = myVCD.get_actions()
    
    for idx,action in actions.items():
        
        name = action['name']
        atype = action['type']
        #print('ACTION = ' + name + ',' + atype)
        ##
        frame_data=[None] * frame_end
        frame_data = fill_frame_data(frame_data, action)
        df['act_'+atype] = frame_data
    df 
    

In [18]:
#df.head()

In [19]:
#df.info()

## Save to CSV

In [20]:
def savefile():
    df.to_csv(outdir +fname+".csv")

In [21]:
def savereport():
    #print fname_act
    actkeys_move_true=[]
    for k,v in move_map.items():
        if v == True:
            actkeys_move_true.append(k)

    ## Print Header
    f = open(outdir + "file_dist.tsv", "w")
    f.write("File\tFrames\t"+"\t".join(sorted(actkeys_move_true)) + "\n")
    for fname in sorted(fname_act.keys()):
        s = fname + "\t" + str(fname_frames[fname])
        for act in sorted(actkeys_move_true):
            if act in fname_act[fname]:
                s += "\t" + str(1)
        f.write(s+"\n")
    f.close()


In [22]:
#frame_start = frame_intervals.'0']['frame_start']
#frame_end = frame_intervals[0]['frame_end']
#print(frame_start, frame_end)


In [23]:
#myVCD = core.vcd('/Users/suresh/dev/dmd/gA_5_s1_2019-03-08T10;57;00+01;00_rgb_ann_distraction.json')


In [24]:
data_dir = './data/'
datafiles = sorted([f for f in listdir(data_dir) if isfile(join(data_dir, f))])
datafiles

for datafile in datafiles:
    #Load a VCD filex
    #global fname
    fname = datafile[:datafile.find('.')]
    print(fname)
    print('frame_end = ' + str(frame_end))
    myVCD = core.VCD(data_dir + fname + ".json")
    
    df=pd.DataFrame()
    parse_frame_intervals()
    movement = [None] * frame_end
    print(str(frame_end))
    print(len(movement))
    
    parse_objects_data()
    parse_contexts()
    parse_actions()
    df['movement'] = pd.Series(movement)
    savefile()

savereport()


gA_1_s1_2019-03-08T09;31;15+01;00_rgb_ann_distraction (1)
frame_end = 0


/usr/local/anaconda3/envs/ds/lib/python3.9/site-packages/vcd/core.py:321: UserWarning: WARNING: Converting VCD 4.3.1 to OpenLABEL 1.0.0. A revision is recommended (specially for transforms and coordinate systems).
  warnings.warn("WARNING: Converting VCD 4.3.1 to OpenLABEL 1.0.0. A revision is recommended (specially for transforms and coordinate systems).")


6776
6776
 recording_context Rainy Car Moving
gA_1_s2_2019-03-08T09;21;03+01;00_rgb_ann_distraction
frame_end = 6776
13640
13640
 recording_context Rainy Car Stopped
gA_1_s3_2019-03-14T14;31;08+01;00_rgb_ann_distraction
frame_end = 13640
1844
1844
 recording_context Rainy Car Stopped
gA_1_s4_2019-03-22T11;49;58+01;00_rgb_ann_distraction
frame_end = 1844
16312
16312
 recording_context Sunny Simulator
gA_2_s1_2019-03-08T10;01;44+01;00_rgb_ann_distraction
frame_end = 16312
7480
7480
 recording_context Sunny Car Moving
gA_2_s2_2019-03-08T09;50;49+01;00_rgb_ann_distraction
frame_end = 7480
14689
14689
 recording_context Rainy Car Stopped
gA_2_s3_2019-03-13T09;23;42+01;00_rgb_ann_distraction
frame_end = 14689
1862
1862
 recording_context Sunny Car Stopped
gA_2_s4_2019-03-22T09;15;55+01;00_rgb_ann_distraction
frame_end = 1862
16219
16219
 recording_context Sunny Simulator
gA_3_s1_2019-03-08T10;27;38+01;00_rgb_ann_distraction
frame_end = 16219
7425
7425
 recording_context Rainy Car Moving
gA_3

In [27]:
#df.head(5000)
df = pd.read_csv('./csv5/gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_distraction.csv')
df
df[df.movement == False]
#df.info()

,Unnamed: 0,frames,driver,driver_fname,driver_name,driver_age,driver_experience,driver_drive_freq,driver_glass,hair_comb,...,act_gaze_on_road/not_looking_road,act_driver_actions/hair_and_makeup,act_driver_actions/texting_right,act_driver_actions/phonecall_right,act_talking/talking,act_hands_using_wheel/none,act_driver_actions/texting_left,act_hands_using_wheel/only_right,act_driver_actions/phonecall_left,movement
0,0,0,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,1,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,2,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,3,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,4,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12410,12410,12410,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
12411,12411,12411,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
12412,12412,12412,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
12413,12413,12413,True,gA_5_s2_2019-03-08T10;46;46+01;00_rgb_ann_dist...,5,33,More than 3 years,Everyday,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
